In [207]:
import urllib
from htmldom import htmldom
import re
import pycurl
import certifi
from io import BytesIO
from urllib.parse import urlencode
import pandas as pd

In [208]:
LABELS={
    'Bad':0, 
    'Neutral':1, 
    'Good':2
}

SMILES={
    'https://tabiturient.ru/img/smile2.png': 'Bad',
    'https://tabiturient.ru/img/smile3.png': 'Neutral',
    'https://tabiturient.ru/img/smile1.png': 'Good',
}

In [209]:
URL = 'https://tabiturient.ru/ajax/ajsliv.php'

def get_list_page_content(vuzid, limit):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, URL)
    c.setopt(c.WRITEDATA, buffer)
    c.setopt(c.CAINFO, certifi.where())
    post_data = {'vuzid': vuzid, 'limit': limit, 'sort': 3}
    postfields = urlencode(post_data)
    c.setopt(c.POSTFIELDS, postfields)    
    c.perform()
    c.close()
    body = buffer.getvalue()
        
    return body.decode('UTF-8')    



In [210]:
def parse_list_page_content(content):
    dom = htmldom.HtmlDom().createDom(content)
    review_element_list = dom.find('div.mobpadd20-2')
    review_url_list = []
    for review in review_element_list:
        review_url_list.append(review.find("a[href]").first().attr('href'))

    return review_url_list
    

In [211]:
def get_review_page_content(review_url):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, review_url)
    c.setopt(c.WRITEDATA, buffer)
    c.setopt(c.CAINFO, certifi.where())
    c.perform()
    c.close()
    body = buffer.getvalue()
        
    return body.decode('UTF-8')    
    

In [212]:
def parse_review(review_text):
    result = {}
    dom = htmldom.HtmlDom().createDom(review_text)
    h1 = dom.find('div.content h1').text()
    if h1=='Такого отзыва не существует':
        return None
    m = re.match('Отзыв №(\\d+)', h1)
    result['id'] = m.group(1)
    result['content'] = dom.find('.content  div.font2').first().text()
    img = dom.find('table.avatarask2 img').first().attr('src')
        
    try:
        result['parent_url'] = dom.find('div.obram a').attr('href')
    except IndexError:
        result['parent_url'] = None
        print('Ссылка на вуз не найдена')
    result['label_name'] = SMILES[img]
    result['label'] = LABELS[result['label_name']]
    return result
    

``Сканирование отзывов одного вуза``

In [216]:
list_page_content=get_list_page_content('mgtu',10)

In [217]:
review_url_list = parse_list_page_content(list_page_content)

In [218]:
data = []

for url in review_url_list:
    content = get_review_page_content(url)
    record = parse_review(content)
    data.append(record)
    print(f"Review ID={record['id']}")


Review ID=48
Review ID=244
Review ID=340
Review ID=374
Review ID=379
Review ID=926
Review ID=1017
Review ID=1356
Review ID=1535
Review ID=1635


In [219]:
df = pd.DataFrame(data)

In [220]:
df

,id,content,parent_url,label_name,label
0,48,Учусь сейчас на первом курсе этого вуза. Выска...,https://tabiturient.ru/vuzu/mgtu,Good,2
1,244,Отзыв о МГТУ.\nБауманка д**уя берет своим назв...,https://tabiturient.ru/vuzu/mgtu,Bad,0
2,340,"Учусь на отраслевом факультете, пришлось посту...",https://tabiturient.ru/vuzu/mgtu,Bad,0
3,374,"Я выпускник Бауманки, бакалавр, факультет инже...",https://tabiturient.ru/vuzu/mgtu,Bad,0
4,379,Я выпускник этого года факультета ИУ. Мое мнен...,https://tabiturient.ru/vuzu/mgtu,Good,2
5,926,Привет! Учусь на направлении &quot;Прикладная ...,https://tabiturient.ru/vuzu/mgtu,Good,2
6,1017,Закончил в этом году бакалавриат ИУ7. \n\nОб у...,https://tabiturient.ru/vuzu/mgtu,Good,2
7,1356,Выпускник вуза 2013 года. Факультет МТ кафедра...,https://tabiturient.ru/vuzu/mgtu,Bad,0
8,1535,"Бред, а не ВУЗ конечно. Пиар на пустом месте. ...",https://tabiturient.ru/vuzu/mgtu,Bad,0
9,1635,Хотела поступить в этот ВУЗ в 2018 г. Как абит...,https://tabiturient.ru/vuzu/mgtu,Neutral,1


In [206]:
df.to_json('hse.jsonl', orient='records', lines=True, force_ascii=False)

``Сканирование всех отзывов``

In [199]:
all_data = []

for id in range(0, 9000):
    url = f"https://tabiturient.ru/sliv/n/?{id}"
    print(url)
    content=get_review_page_content(url)
    record=parse_review(content)
    if record is None:
        print("Не существует")
    else:
        all_data.append(record)
        print("Ok")
        

https://tabiturient.ru/sliv/n/?8309
Не существует
https://tabiturient.ru/sliv/n/?8310
Ok
https://tabiturient.ru/sliv/n/?8311
Ok
https://tabiturient.ru/sliv/n/?8312
Не существует
https://tabiturient.ru/sliv/n/?8313
Не существует
https://tabiturient.ru/sliv/n/?8314
Ok
https://tabiturient.ru/sliv/n/?8315
Ok
https://tabiturient.ru/sliv/n/?8316
Не существует
https://tabiturient.ru/sliv/n/?8317
Ok
https://tabiturient.ru/sliv/n/?8318
Не существует
https://tabiturient.ru/sliv/n/?8319
Не существует
https://tabiturient.ru/sliv/n/?8320
Не существует
https://tabiturient.ru/sliv/n/?8321
Не существует
https://tabiturient.ru/sliv/n/?8322
Ok
https://tabiturient.ru/sliv/n/?8323
Ok
https://tabiturient.ru/sliv/n/?8324
Не существует
https://tabiturient.ru/sliv/n/?8325
Ok
https://tabiturient.ru/sliv/n/?8326
Не существует
https://tabiturient.ru/sliv/n/?8327
Не существует
https://tabiturient.ru/sliv/n/?8328
Не существует
https://tabiturient.ru/sliv/n/?8329
Не существует
https://tabiturient.ru/sliv/n/?8330
Не

In [204]:
df = pd.DataFrame(all_data)

In [206]:
df.to_json('all-reviews.jsonl', orient='records', lines=True, force_ascii=False)